In [57]:
import requests
import pandas as pd
import xmltodict
import json
import re
import numpy as np
import pyecharts
from pyecharts import Bar, Line, Overlap

print("pyecharts version : ", pyecharts.__version__)

pyecharts version :  0.5.11


In [13]:
dong_code = pd.read_csv('data/법정동코드 전체자료.csv', sep = '\t')  #www.code.go.kr 자료
dong_code = dong_code[dong_code['폐지여부']=='존재']

In [15]:
dong_code['법정동코드_re'] = list(str(code)[:5] for code in list(dong_code['법정동코드']))

In [20]:
dong_code_re = dong_code.drop_duplicates(subset=['법정동코드_re'], keep='first').reset_index()

In [7]:
years = ['2015', '2016', '2017', '2018', '2019', '2020']
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

df = pd.DataFrame()
for year in years:
    if year != '2020':
        for month in months:
            data = pd.read_csv('data/아파트 매매/아파트(매매)_' + year + month + '.tsv', delimiter='\t', encoding='euc-kr')
            df = pd.concat([df, data])
    else:
        months2 = ['01', '02', '03']
        for month in months2:
            data = pd.read_csv('data/아파트 매매/아파트(매매)_' + year + month + '.tsv', delimiter='\t', encoding='euc-kr')
            df = pd.concat([df, data])

In [19]:
df.to_csv('data_to_202003.csv', encoding='utf-8-sig')

In [17]:
years = ['2020', '2021']
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

df2 = pd.DataFrame()
for year in years:
    if year == '2020':
        months2 = ['04', '05', '06', '07', '08', '09', '10', '11', '12']
        for month in months2:
            data = pd.read_excel('data/아파트 매매/data_raw/아파트(매매)_실거래가_' + year + month + '.xlsx')
            df2 = pd.concat([df2, data])
    else:
        months2 = ['01', '02', '03', '04', '05',]
        for month in months2:
            data = pd.read_excel('data/아파트 매매/data_raw/아파트(매매)_실거래가_' + year + month + '.xlsx')
            df2 = pd.concat([df2, data])

In [20]:
df2.to_csv('data_to_202105.csv', encoding='utf-8-sig')

In [21]:
df.head(5)

,pnu,시군구,번지,본번,부번,단지명,전용면적,계약연월,계약일,거래금액만원,층,건축년도,도로명
0,4215011500102020000,강원도 강릉시 견소동,202,202,0,송정한신,43.380,201501,9,7300,3,1997,경강로2539번길 8
1,4215011500102020000,강원도 강릉시 견소동,202,202,0,송정한신,84.945,201501,10,13500,9,1997,경강로2539번길 8
2,4215011500102020000,강원도 강릉시 견소동,202,202,0,송정한신,84.945,201501,12,13300,4,1997,경강로2539번길 8
3,4215011500102020000,강원도 강릉시 견소동,202,202,0,송정한신,59.800,201501,14,10400,15,1997,경강로2539번길 8
4,4215011500102020000,강원도 강릉시 견소동,202,202,0,송정한신,84.945,201501,30,11600,1,1997,경강로2539번길 8


In [22]:
df2.head(5)

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일
0,강원도 강릉시 견소동,202,0202,0.0,송정한신,43.380,202004,4,"10,000",9,1997,경강로2539번길 8,NaN
1,강원도 강릉시 견소동,202,0202,0.0,송정한신,84.930,202004,7,"17,300",8,1997,경강로2539번길 8,NaN
2,강원도 강릉시 견소동,202,0202,0.0,송정한신,84.945,202004,10,"14,000",6,1997,경강로2539번길 8,NaN
3,강원도 강릉시 견소동,289,0289,0.0,송정해변신도브래뉴아파트,59.990,202004,7,"13,650",1,2005,경강로2539번길 22,NaN
4,강원도 강릉시 견소동,289,0289,0.0,송정해변신도브래뉴아파트,84.990,202004,11,"23,700",12,2005,경강로2539번길 22,NaN


In [23]:
del  df['pnu']

In [24]:
del  df2['해제사유발생일']

In [38]:
df2 = df2.rename(columns={'전용면적(㎡)':'전용면적', '거래금액(만원)':'거래금액만원', '계약년월':'계약연월'})

In [33]:
df2['거래금액만원'] = [int(x.replace(',','').lstrip()) for x in df2['거래금액만원']]

In [40]:
df_full = pd.concat([df, df2])
df_full.to_csv('아파트매매_2015to2021.csv', encoding='utf-8-sig')

In [25]:
df = pd.read_csv('아파트매매_2015to2021.csv')

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.loc[0, '시군구'].split()[0]

'강원도'

In [26]:
df['juso1'] = [juso.split()[0] for juso in df['시군구']]
df['juso2'] = [juso.split()[1] for juso in df['시군구']]

In [27]:
del df['Unnamed: 0']

In [28]:
import datetime
df['계약연월_re'] = [datetime.datetime.strptime(str(date), '%Y%m') for date in df['계약연월']]

In [45]:
df = pd.read_csv('../data/아파트매매_2015to2021.csv')

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [46]:
del df['Unnamed: 0']

In [47]:
df.head(5)

,시군구,번지,본번,부번,단지명,전용면적,계약연월,계약일,거래금액만원,층,건축년도,도로명,juso1,juso2,계약연월_re
0,강원도 강릉시 견소동,202,202,0.0,송정한신,43.380,201501,9,7300,3,1997,경강로2539번길 8,강원도,강릉시,2015-01-01
1,강원도 강릉시 견소동,202,202,0.0,송정한신,84.945,201501,10,13500,9,1997,경강로2539번길 8,강원도,강릉시,2015-01-01
2,강원도 강릉시 견소동,202,202,0.0,송정한신,84.945,201501,12,13300,4,1997,경강로2539번길 8,강원도,강릉시,2015-01-01
3,강원도 강릉시 견소동,202,202,0.0,송정한신,59.800,201501,14,10400,15,1997,경강로2539번길 8,강원도,강릉시,2015-01-01
4,강원도 강릉시 견소동,202,202,0.0,송정한신,84.945,201501,30,11600,1,1997,경강로2539번길 8,강원도,강릉시,2015-01-01


In [48]:
df['year'] = [date[:4] for date in df['계약연월_re']]

In [49]:
df_2016 = df[df['year']=='2016']
df_2017 = df[df['year']=='2017']
df_2018 = df[df['year']=='2018']
df_2019 = df[df['year']=='2019']
df_2020 = df[df['year']=='2020']
df_2021 = df[df['year']=='2021']

In [55]:
df_2021 = df_2021.reset_index()
del df_2021['index']

In [56]:
len(df_2020)/2

420600.5

In [57]:
df_2020_1 = df_2020[:420600]
df_2020_2 = df_2020[420600:].reset_index()

In [58]:
del df_2020_2['index']

In [59]:
df_2016.to_csv('아파트매매_2016.csv', encoding='utf-8-sig')
df_2017.to_csv('아파트매매_2017.csv', encoding='utf-8-sig')
df_2018.to_csv('아파트매매_2018.csv', encoding='utf-8-sig')
df_2019.to_csv('아파트매매_2019.csv', encoding='utf-8-sig')
df_2020_1.to_csv('아파트매매_2020_1.csv', encoding='utf-8-sig')
df_2020_2.to_csv('아파트매매_2020_2.csv', encoding='utf-8-sig')
df_2021.to_csv('아파트매매_2021.csv', encoding='utf-8-sig')

# 데이터 불러오기

In [2]:
df_2016 = pd.read_csv('아파트매매_2016.csv')
df_2017 = pd.read_csv('아파트매매_2017.csv')
df_2018 = pd.read_csv('아파트매매_2018.csv')
df_2019 = pd.read_csv('아파트매매_2019.csv')
df_2020_1 = pd.read_csv('아파트매매_2020_1.csv')
df_2020_2 = pd.read_csv('아파트매매_2020_2.csv')
df_2021 = pd.read_csv('아파트매매_2021.csv')

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df = pd.concat([df_2016, df_2017, df_2018, df_2019, df_2020_1, df_2020_2, df_2021]).reset_index()

In [4]:
del df['Unnamed: 0']

In [12]:
df_timeseries = df.groupby(by=['계약연월_re']).mean()[['거래금액만원']].reset_index()

In [13]:
df_timeseries.to_csv('아파트매매_거래금액_시계열.csv', encoding='utf-8-sig')

In [15]:
time_region_df = df.groupby(by=['계약연월_re', 'juso1'], dropna=False).mean().round(2)[['거래금액만원']].reset_index()

In [16]:
line = pyecharts.Line("Bar chart")
line.use_theme("dark")
line.add("서울", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='서울특별시']['거래금액만원'], mark_point = ["average", "max", "min"], is_smooth =True, is_symbol_show=False, is_label_show=False)
line.add("부산", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='부산광역시']['거래금액만원'], color = "green",is_smooth =True, is_symbol_show=False)
line.add("대구", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='대구광역시']['거래금액만원'],  color = "blue", is_smooth =True, is_symbol_show=False)
line.add("대전", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='대전광역시']['거래금액만원'], color = "yellow", is_smooth =True, is_symbol_show=False)
line.add("세종", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='세종특별자치시']['거래금액만원'], color = "grey", is_smooth =True, is_symbol_show=False)
line.add("울산", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='울산광역시']['거래금액만원'], color = "white", is_smooth =True, is_symbol_show=False)
line.add("광주", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='광주광역시']['거래금액만원'], color = "orange", is_smooth =True, is_symbol_show=False)
line.add("인천", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='인천광역시']['거래금액만원'],  color = "purple", is_smooth =True, is_symbol_show=False)


line.height = 700
line.width = 1000
line

In [17]:
line = pyecharts.Line("Bar chart")
line.use_theme("dark")
line.add("강원도", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='강원도']['거래금액만원'], mark_line=["average"], is_smooth =True, is_symbol_show=False, is_label_show=False)
line.add("경기도", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='경기도']['거래금액만원'], mark_line=["average"], color = "green",is_smooth =True, is_symbol_show=False)
line.add("경상남도", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='경상남도']['거래금액만원'], mark_line=["average"], color = "blue", is_smooth =True, is_symbol_show=False)
line.add("경상북도", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='경상북도']['거래금액만원'], mark_line=["average"], color = "yellow", is_smooth =True, is_symbol_show=False)
line.add("전라남도", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='전라남도']['거래금액만원'], mark_line=["average"], color = "grey", is_smooth =True, is_symbol_show=False)
line.add("전라북도", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='전라북도']['거래금액만원'], mark_line=["average"], color = "white", is_smooth =True, is_symbol_show=False)
line.add("제주특별자치도", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='제주특별자치도']['거래금액만원'], mark_line=["average"], color = "orange", is_smooth =True, is_symbol_show=False)
line.add("충청남도", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='충청남도']['거래금액만원'], mark_line=["average"], color = "purple", is_smooth =True, is_symbol_show=False)
line.add("충청북도", time_region_df[time_region_df.juso1=='서울특별시']['계약연월_re'], time_region_df[time_region_df.juso1=='충청북도']['거래금액만원'], mark_line=["average"], color = "purple", is_smooth =True, is_symbol_show=False)


line.height = 800
line.width = 1000
line

# 연도별 / 지역별 거래 건수

In [18]:
time_region_df_count = df.groupby(by=['계약연월_re', 'juso1'], dropna=False).count()[['year']].reset_index()

In [19]:
line = pyecharts.Line("Bar chart")
line.use_theme("dark")
line.add("서울", time_region_df_count[time_region_df_count.juso1=='서울특별시']['계약연월_re'], time_region_df_count[time_region_df_count.juso1=='서울특별시']['year'], mark_point = ["average", "max", "min"], is_smooth =True, is_symbol_show=False, is_label_show=False)
line.add("부산", time_region_df[time_region_df_count.juso1=='서울특별시']['계약연월_re'], time_region_df_count[time_region_df_count.juso1=='부산광역시']['year'], color = "green",is_smooth =True, is_symbol_show=False)
line.add("대구", time_region_df_count[time_region_df_count.juso1=='서울특별시']['계약연월_re'], time_region_df_count[time_region_df_count.juso1=='대구광역시']['year'],  color = "blue", is_smooth =True, is_symbol_show=False)
line.add("대전", time_region_df_count[time_region_df_count.juso1=='서울특별시']['계약연월_re'], time_region_df_count[time_region_df_count.juso1=='대전광역시']['year'], color = "yellow", is_smooth =True, is_symbol_show=False)
line.add("세종", time_region_df_count[time_region_df_count.juso1=='서울특별시']['계약연월_re'], time_region_df_count[time_region_df_count.juso1=='세종특별자치시']['year'], color = "grey", is_smooth =True, is_symbol_show=False)
line.add("울산", time_region_df_count[time_region_df_count.juso1=='서울특별시']['계약연월_re'], time_region_df_count[time_region_df_count.juso1=='울산광역시']['year'], color = "white", is_smooth =True, is_symbol_show=False)
line.add("광주", time_region_df_count[time_region_df_count.juso1=='서울특별시']['계약연월_re'], time_region_df_count[time_region_df_count.juso1=='광주광역시']['year'], color = "orange", is_smooth =True, is_symbol_show=False)
line.add("인천", time_region_df_count[time_region_df_count.juso1=='서울특별시']['계약연월_re'], time_region_df_count[time_region_df_count.juso1=='인천광역시']['year'],  color = "purple", is_smooth =True, is_symbol_show=False)


line.height = 700
line.width = 1000
line

# 매매가격별 분석

In [20]:
df_lt_9 = df[df['거래금액만원']>=90000].reset_index()

In [21]:
len(df_lt_9)/ len(df)

0.03978558283330513

In [22]:
df_lt_9_gb = df_lt_9.groupby(by=['계약연월_re'], dropna=False).count().round(2)[['거래금액만원']].reset_index()

In [23]:
line = pyecharts.Line("Bar chart")
line.use_theme("dark")
line.add("9억원 이상 아파트 매매 건수", df_lt_9_gb['계약연월_re'], df_lt_9_gb['거래금액만원'], mark_point = ["average", "max", "min"], is_smooth =True, is_symbol_show=False, is_label_show=False)

line.height = 500
line.width = 1000
line

In [24]:
#실거래가 9억원 이하
df['lesst_9'] = [1 if x >= 90000 else 0 for x in df['거래금액만원']]

In [25]:
lesst_9_percentage = df.groupby(by=['계약연월_re'], dropna=False).mean().round(2)[['lesst_9']].reset_index()

In [26]:
line = pyecharts.Line("Bar chart")
line.use_theme("dark")
line.add("9억원 이하 아파트 매매 비율", lesst_9_percentage['계약연월_re'], lesst_9_percentage['lesst_9'], mark_point = ["average", "max", "min"], is_smooth =True, is_symbol_show=False, is_label_show=False)

line.height = 500
line.width = 1000
line

In [30]:
df_seoul = df[df['juso1']=='서울특별시']

In [32]:
lt_9_pct_seoul = df_seoul.groupby(by=['계약연월_re'], dropna=False).mean().round(2)[['lesst_9']].reset_index()

In [33]:
line = pyecharts.Line("Bar chart")
line.use_theme("dark")
line.add("서울 9억원 이상 아파트 매매 비율", lt_9_pct_seoul['계약연월_re'], lt_9_pct_seoul['lesst_9'], mark_point = ["average", "max", "min"], is_smooth =True, is_symbol_show=False, is_label_show=False)

line.height = 500
line.width = 1000
line

In [34]:
#실거래가 6억원 이하
df['lesst_6'] = [1 if x <= 60000 else 0 for x in df['거래금액만원']]

In [35]:
df_seoul = df[df['juso1']=='서울특별시']
lesst_6_pct_seoul = df_seoul.groupby(by=['계약연월_re'], dropna=False).mean().round(2)[['lesst_6']].reset_index()

In [36]:
line = pyecharts.Line("Bar chart")
line.use_theme("dark")
line.add("서울 6억원 이하 아파트 매매 비율", lesst_6_pct_seoul['계약연월_re'], lesst_6_pct_seoul['lesst_6'], mark_point = ["average", "max", "min"], is_smooth =True, is_symbol_show=False, is_label_show=False)

line.height = 500
line.width = 1000
line

In [37]:
#전용면적 85이하
df['lesst_85_size'] = [1 if x <= 85 else 0 for x in df['전용면적']]

In [38]:
df_seoul = df[df['juso1']=='서울특별시']
df_seoul_lt_85 = df_seoul[df_seoul['lesst_85_size']==1]
lt_size85_seoul = df_seoul_lt_85.groupby(by=['계약연월_re'], dropna=False).mean().round(2)[['거래금액만원']].reset_index()

line = pyecharts.Line("Bar chart")
line.use_theme("dark")
line.add("서울 전용면적 85 이하 아파트 실거래가 매월 평균", lt_size85_seoul['계약연월_re'], lt_size85_seoul['거래금액만원'], mark_point = ["average", "max", "min"], is_smooth =True, is_symbol_show=False, is_label_show=False)

line.height = 500
line.width = 1000
line

In [39]:
lesst_6_pct_seoul_85size = df_seoul_lt_85.groupby(by=['계약연월_re'], dropna=False).mean().round(2)[['lesst_6']].reset_index()


line = pyecharts.Line("Bar chart")
line.use_theme("dark")
line.add("서울 전용면적 85 이하 아파트 6억원 미만 비율", lesst_6_pct_seoul_85size['계약연월_re'], lesst_6_pct_seoul_85size['lesst_6'], mark_point = ["average", "max", "min"], is_smooth =True, is_symbol_show=False, is_label_show=False)

line.height = 500
line.width = 1000
line

In [51]:
df_seoul = df_seoul.reset_index()

In [67]:
del df_seoul['level_0']
del df_seoul['index']

In [68]:
df_seoul.head(5)

,시군구,번지,본번,부번,단지명,전용면적,계약연월,계약일,거래금액만원,층,건축년도,도로명,juso1,juso2,계약연월_re,year,lesst_85_size,lesst_9,lesst_6,단지명_re
0,서울특별시 강남구 개포동,141,141,0.0,개포주공 1단지,49.56,201601,4,88000,2,1982,선릉로 7,서울특별시,강남구,2016-01-01,2016,1,0,0,개포주공 1단지
1,서울특별시 강남구 개포동,141,141,0.0,개포주공 1단지,35.64,201601,9,64000,4,1982,개포로 310,서울특별시,강남구,2016-01-01,2016,1,0,0,개포주공 1단지
2,서울특별시 강남구 개포동,141,141,0.0,개포주공 1단지,49.56,201601,11,90000,3,1982,선릉로 7,서울특별시,강남구,2016-01-01,2016,1,1,0,개포주공 1단지
3,서울특별시 강남구 개포동,141,141,0.0,개포주공 1단지,50.64,201601,14,89000,3,1982,개포로 310,서울특별시,강남구,2016-01-01,2016,1,0,0,개포주공 1단지
4,서울특별시 강남구 개포동,141,141,0.0,개포주공 1단지,50.64,201601,20,87300,2,1982,개포로 310,서울특별시,강남구,2016-01-01,2016,1,0,0,개포주공 1단지


In [69]:
df_seoul['juso3'] = [juso.split()[2] for juso in df_seoul['시군구']]

In [59]:
regex = "\(.*\)|\s-\s.*"
df_seoul['단지명_re'] = [re.sub(regex, '', x) for x in df_seoul['단지명']]

In [89]:
seoul_gu_price = df_seoul.groupby(by=['juso2', '계약연월_re'], dropna=False).mean().round(1)[['거래금액만원']].reset_index()

In [82]:
seoul_gu_price.head(5)

,juso2,계약연월_re,거래금액만원
0,강남구,2016-01-01,98086.767273
1,강남구,2016-02-01,100002.089552
2,강남구,2016-03-01,94457.392739
3,강남구,2016-04-01,103690.061414
4,강남구,2016-05-01,109956.265985


In [100]:
gu_list = list(set(seoul_gu_price['juso2']))

line = pyecharts.Line("Bar chart")
line.use_theme("dark")


for gu in gu_list:
    line.add( gu , 
             seoul_gu_price[seoul_gu_price['juso2']==gu]['계약연월_re'], 
             seoul_gu_price[seoul_gu_price['juso2']==gu]['거래금액만원'], 
             #mark_point = [ "max"], 
             is_smooth =True, 
             is_symbol_show=False, 
             is_label_show=False)

line.height = 700
line.width = 900
line

In [103]:
df_seoul.groupby(by=['juso2', 'year'], dropna=False).count()[['거래금액만원']].reset_index()

,juso2,year,거래금액만원
0,강남구,2016,6758
1,강남구,2017,7264
2,강남구,2018,3515
3,강남구,2019,4620
4,강남구,2020,3762
...,...,...,...
145,중랑구,2017,3156
146,중랑구,2018,2886
147,중랑구,2019,2346
148,중랑구,2020,2812
